In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset

In [2]:
train_ds, test_ds = read_dataset("ArrowHead")
train_dl = DataLoader(train_ds, batch_size=2, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=2, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = 2


Loading the ArrowHead dataset...
The dataset ArrowHead was loaded.


In [3]:
dtcr_model = DTCRModel(config)


In [4]:
encoder = dtcr_model.encoder
encoder


DRNN(
  (cells): Sequential(
    (0): GRU(1, 100)
    (1): GRU(100, 50)
    (2): GRU(50, 50)
  )
)

In [25]:
for sample_data, sample_label in train_dl:
    out, hidden = encoder(sample_data)
    break


torch.Size([2, 251, 1])


In [24]:
out

torch.Size([1, 251, 100])